## 2008-2010に大きく株価を下げた会社をピックアップ
+ [リーマン・ショック - Wikipedia](https://ja.wikipedia.org/wiki/%E3%83%AA%E3%83%BC%E3%83%9E%E3%83%B3%E3%83%BB%E3%82%B7%E3%83%A7%E3%83%83%E3%82%AF)
    + 2007年のアメリカ合衆国の住宅バブル崩壊をきっかけとして、サブプライム住宅ローン危機を始め、プライムローン、オークション・レート証券、カードローン関連債券など多分野にわたる資産価格の暴落が起こっていた。


## 悪い会社リスト
+ Reginal Bank: {'BPOP', 'FHN', 'FITB', 'HBAN', 'RF', 'WBS'}
+ Savings & Cooperative Banks: {'FBC', 'NWBI', 'PBCT', 'WAFD'}
+ Specialty Finance: {'CIT', 'HTH'} ← Morgage 
+ Credit Services : {'AXP', 'COF', 'SLM'}
+ Insurance - Property & Casualty: {'CNA', 'MBI', 'STC'}


In [ ]:
import numpy as np
import pandas as pd 

from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.filters import QTradableStocksUS, StaticAssets

from quantopian.pipeline.data import USEquityPricing

from quantopian.pipeline.data.morningstar import Fundamentals

def make_pipeline():
    
    base_universe = QTradableStocksUS()
    myassets = StaticAssets(symbols(['BPOP', 'FHN', 'FITB', 'HBAN', 'RF', 'WBS', 'FBC', 'NWBI', 'PBCT', 'WAFD', 'CIT', 'HTH', 'AXP', 'COF', 'SLM','CNA', 'MBI', 'STC']))
    
    close_price = USEquityPricing.close.latest
    
    # sector code 
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    mortgage_and_consumerloans = Fundamentals.mortgage_and_consumerloans.latest
    mortgage_loan = Fundamentals.mortgage_loan.latest
    total_assets = Fundamentals.total_assets.latest
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    return Pipeline(
        columns = {
            'close_price': close_price,
            'morningstar_industry_code':morningstar_industry_code, 
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'mortgage_and_consumerloans':mortgage_and_consumerloans, 
            'mortgage_loan':mortgage_loan, 
            'financial_health_grade':financial_health_grade,
            
        },
        
        screen=base_universe
    )
 
    
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2007-01-01',
    end_date='2020-05-01'
)

pipeline_output.tail(10)


## Fランクの株を見つけてみる

In [ ]:
## F
rank_F = pipeline_output.loc["2020-05-01"]["financial_health_grade"][pipeline_output.loc["2020-05-01"]["financial_health_grade"]=="F"].index.get_level_values(1).tolist()

In [ ]:
idx = pd.IndexSlice
pipeline_output.loc[idx[:,rank_F],:]["financial_health_grade"].unstack()



In [ ]:
# 10320050 : Banks - Regional - US
df = rank_F[pipeline_output["morningstar_industry_code"]==10326060]

In [ ]:
# asset name を見たい場合
df.loc["2007-01-03"].index.get_level_values(level=1).to_series().apply(lambda x: x.asset_name)

In [ ]:
# 2007-2010 の株価推移を確認
df_cumsum = df["close_price"].unstack()["2007":"2010"].dropna(axis=1,how="all").fillna(method="ffill").pct_change().iloc[1:].cumsum()


In [ ]:
# describe してデータを取得
# 当該期間中に上場したり、非上場になったりしてデータが揃っていないので１つずつForLoopで回している
d = dict()
for sym in df_cumsum.columns:
    d[sym] = df_cumsum[sym].dropna().describe()
    

In [ ]:
# まだ生きている会社のリストを取得
alives = pipeline_output.xs("2020-05-01", level=0).index.to_series().apply(lambda x: x.symbol).tolist()


In [ ]:
# 調査期間中に、最も株価が落ちた会社で、現在も上場している会社を取得
set(pd.DataFrame(d).T.sort_values(by="min").head(5).index.to_series().apply(lambda x: x.symbol).tolist()) & set(alives)


## ここ数年でmortgage loan の比率が total assets に対して増えている会社を探す

+ `mortgage_loan`: This is a lien on real estate to protect a lender. This item is typically available for bank industry.


In [ ]:
def make_pipeline():
    
    myassets = StaticAssets(symbols(['BPOP', 'FHN', 'FITB', 'HBAN', 'RF', 'WBS', 'FBC', 'NWBI', 'PBCT', 'WAFD', 'CIT', 'HTH', 'AXP', 'COF', 'SLM','CNA', 'MBI', 'STC']))
    
    close_price = USEquityPricing.close.latest
    
    # sector code 
    morningstar_industry_group_code = Fundamentals.morningstar_industry_group_code.latest
    morningstar_industry_code = Fundamentals.morningstar_industry_code.latest
    mortgage_and_consumerloans = Fundamentals.mortgage_and_consumerloans.latest
    mortgage_loan = Fundamentals.mortgage_loan.latest
    total_assets = Fundamentals.total_assets.latest
    stockholders_equity = Fundamentals.stockholders_equity.latest
    financial_health_grade = Fundamentals.financial_health_grade.latest
    
    return Pipeline(
        columns = {
            'close_price': close_price,
            'morningstar_industry_code':morningstar_industry_code, 
            'morningstar_industry_group_code':morningstar_industry_group_code,
            'mortgage_and_consumerloans':mortgage_and_consumerloans, 
            'mortgage_loan':mortgage_loan, 
            'total_assets':total_assets,
            'stockholders_equity':stockholders_equity,
            'financial_health_grade':financial_health_grade,
        },
        screen=myassets
    )
 
    
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date='2016-01-01',
    end_date='2020-05-01'
)

pipeline_output.tail(10)


In [ ]:
pipeline_output["mortgage_loan_asset_ratio"] = pipeline_output["mortgage_loan"]/pipeline_output["total_assets"]
pipeline_output["stockholders_equity_ratio"] = pipeline_output["stockholders_equity"]/pipeline_output["total_assets"]

In [ ]:
pipeline_output["stockholders_equity_ratio"].unstack().iloc[-1].sort_values()

In [ ]:
pipeline_output["mortgage_loan_asset_ratio"].unstack().plot()

In [ ]:
pipeline_output.xs